## Import libraries

In [1]:
import pandas as pd
import numpy as np

## Set filenames

In [2]:
orders=pd.read_csv("Data/orders.csv")
customers=pd.read_csv("Data/customers_p1.csv")
vendors=pd.read_csv("Data/vendors_p1.csv")

C:\Users\dira9\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (15,16,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## For every customer details, loading 100 vendor details

In [3]:
## Reduce memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
customers=reduce_mem_usage(customers)
vendors=reduce_mem_usage(vendors)
orders=reduce_mem_usage(orders)

Mem. usage decreased to  1.53 Mb (78.9% reduction)
Mem. usage decreased to  0.01 Mb (85.0% reduction)
Mem. usage decreased to 17.55 Mb (34.6% reduction)


In [5]:
cust_ven = customers.assign(key=1).merge(vendors.assign(key=1), on='key').drop('key',axis=1)

In [6]:
cust_ven.shape

(5950300, 103)

In [7]:
cust_ven.columns.values

array(['dob', 'cust_status', 'cust_verified', 'customer_id',
       'cust_location_number', 'Cust_created_days',
       'cust_location_type_Home', 'cust_location_type_Other',
       'cust_location_type_Work', 'gender_female', 'gender_male',
       'cust_lan_long_0', 'cust_lan_long_1', 'cust_lan_long_2',
       'cust_lan_long_3', 'cust_lan_long_4', 'vendor_id',
       'delivery_charge', 'serving_distance', 'is_open',
       'prepration_time', 'discount_percentage', 'ven_status',
       'ven_verified', 'vendor_rating', 'Ven_created_days', 'american',
       'arabic', 'asian', 'bagels', 'biryani', 'breakfast', 'burgers',
       'cafe', 'cakes', 'chinese', 'churros', 'coffee', 'combos',
       'crepes', 'desserts', 'dimsum', 'donuts', 'family_meal',
       'fatayers', 'free_delivery', 'fresh_juices', 'fries',
       'frozen_yoghurt', 'grills', 'healthy_food', 'hot_chocolate',
       'hot_dogs', 'ice_creams', 'indian', 'italian', 'japanese', 'karak',
       'kebabs', 'kids_meal', 'kushari',

In [8]:
cust_ven['train_CID X LOC_NUM X VENDOR'] = cust_ven['customer_id'] + " X " + cust_ven['cust_location_number'].astype(str) + " X " + cust_ven['vendor_id'].astype(str)

In [9]:
cust_ven['train_CID X LOC_NUM X VENDOR'].head()

0     000THBA X 0 X 4
1    000THBA X 0 X 13
2    000THBA X 0 X 20
3    000THBA X 0 X 23
4    000THBA X 0 X 28
Name: train_CID X LOC_NUM X VENDOR, dtype: object

## Order dataset

In [10]:
orders.head()

,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.601562,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.703125,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.398438,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.101562,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.203125,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [11]:
orders.columns.values

array(['akeed_order_id', 'customer_id', 'item_count', 'grand_total',
       'payment_mode', 'promo_code', 'vendor_discount_amount',
       'promo_code_discount_percentage', 'is_favorite', 'is_rated',
       'vendor_rating', 'driver_rating', 'deliverydistance',
       'preparationtime', 'delivery_time', 'order_accepted_time',
       'driver_accepted_time', 'ready_for_pickup_time', 'picked_up_time',
       'delivered_time', 'delivery_date', 'vendor_id', 'created_at',
       'LOCATION_NUMBER', 'LOCATION_TYPE', 'CID X LOC_NUM X VENDOR'],
      dtype=object)

In [12]:
orders['CID X LOC_NUM X VENDOR'].head()

0    92PEE24 X 0 X 105
1    QS68UD8 X 0 X 294
2     MB7VY5F X 0 X 83
3     KDJ951Y X 0 X 90
4     BAL0RVT X 0 X 83
Name: CID X LOC_NUM X VENDOR, dtype: object

## Create target variable 

#### Compare CID X LOC_NUM X VENDOR in order and cust_loc_ven, if there is a match, then create target as 1 else 0

In [13]:
#Creating target in orders where all the values are 1
orders['Target']=1

In [14]:
orders_extract=orders[['CID X LOC_NUM X VENDOR','Target']]

In [15]:
orders_extract.shape

(135303, 2)

#### Delete duplicate order records

In [16]:
orders_extract.drop_duplicates(subset='CID X LOC_NUM X VENDOR',keep='first',inplace=True)

C:\Users\dira9\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
orders_extract.shape

(80142, 2)

In [18]:
cust_order_ven=pd.merge(cust_ven,orders_extract,how="left",left_on='train_CID X LOC_NUM X VENDOR',
                      right_on='CID X LOC_NUM X VENDOR')

In [19]:
cust_order_ven.shape

(5950300, 106)

In [20]:
#we can delete the cid*loc*target from orders in the new datset
del cust_order_ven['CID X LOC_NUM X VENDOR']
cust_order_ven.drop(['customer_id','cust_location_number','vendor_id','dob','cust_status','cust_verified','OpenHour'],
                    axis=1, inplace=True)

In [21]:
cust_order_ven

,Cust_created_days,cust_location_type_Home,cust_location_type_Other,cust_location_type_Work,gender_female,gender_male,cust_lan_long_0,cust_lan_long_1,cust_lan_long_2,cust_lan_long_3,...,vendor_category_en_Restaurants,vendor_category_en_Sweets & Bakes,device_type_1,device_type_3,timegroup_early(5-10.59am),timegroup_evening(17-22.59pm),timegroup_latenight(23-4.59am),timegroup_normal(11-16.59pm),train_CID X LOC_NUM X VENDOR,Target
0,595.0,0,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,1,000THBA X 0 X 4,NaN
1,595.0,0,0,0,0,1,0,1,0,0,...,1,0,0,1,1,0,0,0,000THBA X 0 X 13,NaN
2,595.0,0,0,0,0,1,0,1,0,0,...,1,0,0,1,1,0,0,0,000THBA X 0 X 20,NaN
3,595.0,0,0,0,0,1,0,1,0,0,...,1,0,0,1,1,0,0,0,000THBA X 0 X 23,NaN
4,595.0,0,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,1,000THBA X 0 X 28,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5950295,NaN,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,RGN74TG X 0 X 849,NaN
5950296,NaN,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,RGN74TG X 0 X 855,NaN
5950297,NaN,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,RGN74TG X 0 X 856,NaN
5950298,NaN,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,RGN74TG X 0 X 858,NaN


In [22]:
len(cust_order_ven['Target'][cust_order_ven['Target']==1])
#We have 135303 target 1 from the orders dataset

80142

In [23]:
# fill the Target column for non matching records with 0 (non matching with orders file) 
cust_order_ven['Target']=cust_order_ven['Target'].fillna(0)

In [24]:
len(cust_order_ven['Target'][cust_order_ven['Target']==0])

5870158

In [25]:
cust_order_ven.columns.values

array(['Cust_created_days', 'cust_location_type_Home',
       'cust_location_type_Other', 'cust_location_type_Work',
       'gender_female', 'gender_male', 'cust_lan_long_0',
       'cust_lan_long_1', 'cust_lan_long_2', 'cust_lan_long_3',
       'cust_lan_long_4', 'delivery_charge', 'serving_distance',
       'is_open', 'prepration_time', 'discount_percentage', 'ven_status',
       'ven_verified', 'vendor_rating', 'Ven_created_days', 'american',
       'arabic', 'asian', 'bagels', 'biryani', 'breakfast', 'burgers',
       'cafe', 'cakes', 'chinese', 'churros', 'coffee', 'combos',
       'crepes', 'desserts', 'dimsum', 'donuts', 'family_meal',
       'fatayers', 'free_delivery', 'fresh_juices', 'fries',
       'frozen_yoghurt', 'grills', 'healthy_food', 'hot_chocolate',
       'hot_dogs', 'ice_creams', 'indian', 'italian', 'japanese', 'karak',
       'kebabs', 'kids_meal', 'kushari', 'lebanese', 'manakeesh',
       'mandazi', 'mexican', 'milkshakes', 'mishkak', 'mojitos',
       'mojito

In [26]:
# drop columns with NaN values
cust_order_ven.dropna(axis='columns',inplace=True)

In [27]:
# make target variable as int
cust_order_ven ['Target'] = cust_order_ven ['Target'].astype(int)

In [28]:
cust_order_ven=reduce_mem_usage(cust_order_ven)

Mem. usage decreased to 680.96 Mb (2.4% reduction)


In [29]:
#convert to csv file
cust_order_ven.to_csv("cust_order_ven_p2.csv",index=False)

In [30]:
cust_order_ven_sample20 = cust_order_ven.sample(frac=0.2)

In [31]:
cust_order_ven_sample20

,cust_location_type_Home,cust_location_type_Other,cust_location_type_Work,gender_female,gender_male,cust_lan_long_0,cust_lan_long_1,cust_lan_long_2,cust_lan_long_3,cust_lan_long_4,...,vendor_category_en_Restaurants,vendor_category_en_Sweets & Bakes,device_type_1,device_type_3,timegroup_early(5-10.59am),timegroup_evening(17-22.59pm),timegroup_latenight(23-4.59am),timegroup_normal(11-16.59pm),train_CID X LOC_NUM X VENDOR,Target
3793547,1,0,0,0,1,1,0,0,0,0,...,1,0,0,1,0,0,0,1,NKK2N8J X 0 X 197,0
5462106,1,0,0,0,1,1,0,0,0,0,...,1,0,0,1,0,0,0,1,Y09JVZB X 1 X 43,0
880681,0,0,1,0,1,0,1,0,0,0,...,1,0,0,1,1,0,0,0,5G28UMA X 1 X 547,0
2638614,0,0,0,0,1,0,1,0,0,0,...,1,0,0,1,0,0,0,1,GDJ5Q3V X 0 X 79,0
2646447,0,0,1,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,1,GEZ00L3 X 3 X 197,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571365,0,0,0,0,1,0,1,0,0,0,...,1,0,0,1,1,0,0,0,9QVPIV3 X 0 X 294,0
2932838,0,0,0,0,1,0,1,0,0,0,...,1,0,0,1,1,0,0,0,IAK7GJ2 X 2 X 176,0
3365985,0,0,0,0,1,0,1,0,0,0,...,1,0,0,1,0,0,0,1,KYKSU43 X 0 X 578,0
2483173,0,0,0,0,1,0,1,0,0,0,...,1,0,0,1,0,0,0,1,FFLF1VQ X 0 X 356,0


In [32]:
#convert to csv file
cust_order_ven_sample20.to_csv("cust_order_ven__sample20_p2.csv",index=False)